In [1]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import json
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import time
import re

from functions import (get_sector, changes_from_press, get_content_header,
                       get_df, normalise_to_index, save_data)



In [2]:

year = datetime.today().year

driver = webdriver.Firefox()
driver.get('https://shareholder.ford.com/Investors/news/default.aspx')

data = []

ticker = 'f'
index_ticker = '^GSPC'


sector = get_sector(ticker)
stock_hist = get_df(ticker)

index_hist = get_df(index_ticker)

year = datetime.today().year

while True:

    select = Select(driver.find_element(By.ID, "newsYear"))
    if year == 2013:
        break
    # select by value 
    try:
        select.select_by_value(str(year))
    except:
        break

    time.sleep(2)

    html = driver.page_source


    soup = BeautifulSoup(html) 

    articles = soup.find_all('div',attrs={'class':'module_item'})
    
    for article in articles:
        date = article.find('div', attrs={'class':'module_date-time'}).text.lstrip()
        title = article.find('div', attrs={'class':'module_headline'}).text
        url = article.find('a', attrs={'class':'module_headline-link'}).get('href')
        url = f"https://shareholder.ford.com{url}"
        
        date = date.lstrip().rstrip()    # remove starting and trailing whitespaces
        title = title.lstrip().rstrip() 
        title = re.sub("\s\s+", " ", title)

        date = datetime.strptime(date, '%B %d, %Y').date()
        pct_change = changes_from_press(stock_hist, date, 1)
        
        content = get_content_header()
        
        if pct_change != None:
            index_price, norm_price = normalise_to_index(pct_change, date, index_hist)
        else:
            index_price, norm_price = (None, None)

        data.append([date, title, url, content, pct_change, index_price, norm_price])
        
    year = year - 1

driver.close()

save_data(data, columns, ticker, sector)

TypeError: get_content_header() missing 3 required positional arguments: 'url', 'tag', and 'attrs'

In [3]:
requests.get('https://s201.q4cdn.com/693218008/files/doc_news/2023/05/ms-sustainable-summit-advisory-may-25-2023_final.pdf').text

'%PDF-1.6\r%����\r\n154 0 obj\r<</Linearized 1/L 215514/O 156/E 181058/N 2/T 215166/H [ 556 253]>>\rendobj\r             \r\n188 0 obj\r<</DecodeParms<</Columns 5/Predictor 12>>/Filter/FlateDecode/ID[<8016692CFB66774FAADAF660C97E1F38><033EEECA9369A346ACEBFDBCE52B48BA>]/Index[154 53]/Info 153 0 R/Length 146/Prev 215167/Root 155 0 R/Size 207/Type/XRef/W[1 3 1]>>stream\r\nh�bbd`\x10``b``�\x04�\x0c�A$�M\x10�:\x11�~\x07&�@$�j�J\r0�\x02\x16�\tf��I\x07��"��\x10\x10Y \x05$\x19��\x06��\x04��A��\x16\x10��\x04"���lM��\x00�\x08c\x19�L�\x00�\x17\x01�z�\x11&0ǀm�\x0b\x16�\x0c"�O��\x0b��?�\x15\x0cL@\x1fi����HC�?\x03S�\x07�\x00\x03\x00>S\x17�\r\nendstream\rendobj\rstartxref\r\n0\r\n%%EOF\r\n        \r\n206 0 obj\r<</C 159/Filter/FlateDecode/I 181/Length 165/S 64>>stream\r\nh�b```e``:\x0c"�61\x081 �\x10\x033\x03\x1b\x03\x0b\x03�\r�i&@�\x0e��@s�a\x07\x06�\x06 \x02\x0b\x02\x01\x1b\x03��\x17 �\x00Ċ`�7\x0c\x02\x0co��221F1�g�a�`Z�d�\x14´�1#_o�yƳ��J\'\x18�|�\x19l��\x18��u\x185t�3�e*`��p\\�\x0e�V�.�3�\x19��\x1

In [7]:
from PyPDF2 import PdfReader
reader = PdfReader('https://s201.q4cdn.com/693218008/files/doc_news/2023/05/ms-sustainable-summit-advisory-may-25-2023_final.pdf')

OSError: [Errno 22] Invalid argument: 'https://s201.q4cdn.com/693218008/files/doc_news/2023/05/ms-sustainable-summit-advisory-may-25-2023_final.pdf'